# MBI HW1 for CS students (academic year 2023/24, winter semester)

Add your code to this template.

# Module imports and function definitions

Below we import some useful modules and define several functions including those listed in the assignment. Do not modify this part.

In [ ]:
# imports
import io
import os
import re
import gc
import sys
import random
import heapq
import logging
logging.basicConfig(level=logging.WARNING)
logger = logging.getLogger('mbi').setLevel(logging.DEBUG)
import itertools
import hashlib

import requests
from numba import njit
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
from tqdm.notebook import trange, tqdm

!pip install biopython
from Bio import SeqIO



In [ ]:
def _download_file(url):
  """Auxiliary function to download a file from an url and return it as a string."""
  logger = logging.getLogger('mbi')
  logger.debug(f"Downloading from '{url}'...")
  sys.stderr.flush()
  response = requests.get(url)
  if response.status_code != 200:
    raise Exception(f"Cannot access '{url}'! Status code {response.status_code}")
  return response.text


def read_fasta(url=None, filename=None):
  """Function reads sequences in a FASTA format and returns them as a list of strings consisting of letters A,C,G,T.
  It also returns a list of descriptions of these sequences. If filename is given and it exists,
  sequences are read from the file. Otherwise they are read from the URL and written to the file."""
  logger = logging.getLogger('mbi')

  # open file or url
  handle = None
  output_handle = None
  if filename is not None and os.path.isfile(filename) \
  and os.path.getsize(filename) > 0:
    handle = open(filename, 'r')
  elif url is not None:
    from urllib.request import urlopen
    handle = io.TextIOWrapper(urlopen(url), encoding='utf-8')
    if filename is not None:
      output_handle = open(filename, "w")

  real_genomes, real_genome_descriptions = [], []
  for gnum, genome in enumerate(SeqIO.parse(handle, "fasta")):
    if output_handle is not None:
      SeqIO.write(genome, output_handle, "fasta")
    sequence = str(genome.seq).replace('N', '')
    if not re.match(r'\A[ACGT]+\Z', sequence):
      raise ValueError(f"Bad genome characters in the input file")
    description = genome.description
    real_genomes.append(sequence)
    real_genome_descriptions.append(description)
    logger.debug(f"Parsed genome #{gnum}: {description[:70]}...")

  if output_handle is not None:
    output_handle.close()
  handle.close()

  return real_genomes, real_genome_descriptions


@njit
def c(x: str) -> str:
  """Returns complement of a single DNA base"""
  if x == 'A':
    return 'T'
  elif x == 'T':
    return 'A'
  elif x == 'C':
    return 'G'
  elif x == 'G':
    return 'C'
  else:
    return ""


@njit
def _reverse_complement(seq: str) -> str:
  """Returns reverse complement (opposite strand) of a DNA sequence"""
  return "".join([c(seq[i]) for i in range(len(seq)-1, -1, -1)])


@njit
def canonical(kmer: str) -> str:
  """Returns a canonical k-mer for the input k-mer. """
  comp_kmer = _reverse_complement(kmer)
  if kmer <= comp_kmer:
    return kmer
  else:
    return comp_kmer


@njit
def full_kmer_set(sequence: str, k: int) -> set[str]:
  """Function gets a string consisting of letters A,C,G,T representing a single genome and integer k and returns a Python set of all canonical k-mers of the input string."""
  result = set()
  for start in range(len(sequence) - k + 1):
    result.add(canonical(sequence[start:start+k]))
  return result


@njit
def mutate(sequence: str, p: float) -> str:
  """Function gets a string consisting of letters A,C,G,T representing a single genome and a mutation probability p (a real number between 0 and 1).
  For each base of the input string it will decide with probability p to mutate it or 1-p to leave it as it was.
  If the base is mutated, it is replaced by a randomly chosen base from the remaining three."""
  others = {x: "ACGT".replace(x, "") for x in "ACGT"}
  result = "".join([x if random.random() > p else others[x][random.randint(0, 2)] for x in sequence])
  return result

def shash(seq: str) -> int:
  """A hash function to be used for hashing k-mers in minimizers and MinHash"""
  hasher = hashlib.blake2b(salt=b'')
  hasher.update(seq.encode(encoding='utf-8'))
  d = hasher.digest()
  result = int.from_bytes(d[:5], byteorder='little')
  return result


# testing
print(f"{[x + '->' + c(x) for x in 'ACGT']}")
print(f"{_reverse_complement('AGTG')=}")
print(f"{_reverse_complement('CACT')=}")
print(f"{canonical('AGTG')=}")
print(f"{canonical('CACT')=}")
print(f"{list(full_kmer_set('CCAAGGTCCATC', k=3))=}")
print(f"{[mutate('AAAACCCCGGGGTTTT', p=0.25) for _ in range(5)]=}")
print(f"{shash('A' * 13)=}, {shash('ACTACTACTACTG')=} {shash('T'*12 + 'A')=}")


## Downloading genomes

This part will download bacterial genomes and create mutated versions of genome 0 as described in the assignment. When you run this cell first time, the sequences will be downloaded from the internet and saved to `bacteria.fasta` file. Subsequently, this file will be read instead.

In [ ]:
url = "http://compbio.fmph.uniba.sk/vyuka/mbi-data/du1/bacteria.fasta"
real_genomes, real_genome_descriptions = read_fasta(url=url, filename="bacteria.fasta")
if len(real_genomes) != 9:
  raise Exception("Some problem with reading sequences, perhaps delete bacteria.fasta file")
gc.collect()

In [ ]:
mutation_probabilities = [0.01, 0.05, 0.1, 0.2, 0.3, 0.4]
random.seed(47)
mutated_genomes = [mutate(real_genomes[0], p) for p in mutation_probabilities]
gc.collect()

## Task 1: Jaccard similarity

Implement function `jaccard` below and run it on the real genomes as specified in the assignment.

In [ ]:
def jaccard(first_set: set[str], list_of_sets: list[set[str]]) -> list[float]:
  """Function gets one set and a list of n sets
  and computes a vector of n Jaccard similarities of the first set compared each of the sets in the list. """
  pass


## Task 2: Minimizers

Implement function `minimizer_set` below and run it on a portion of genome 0 as specified in the assignment.

In [ ]:
def minimizer_set(sequence: str, k: int, w: int) -> set[str]:
  """Function gets a string consisting of letters A,C,G,T and two parameters k and w
  and returns a Python set of all minimizers of the string. """
  pass

## Task 3: MinHash

Implement function `minhash_set` below and run it on a portion of genome 0 as specified in the assignment.

In [ ]:
def minhash_set(sequence : str, k : int, m : int)-> set[str]:
  """Function gets a string consisting of letters A,C,G,T and two integers k and m
  and returns the minhash set of the string."""
  pass

Implement function `minhash_jaccard` below and run it on all real genomes as specified in the assignment.

In [ ]:
def minhash_jaccard(first_set: set[str], list_of_sets: list[set[str]]) -> list[float]:
  pass

## Task 4: Accuracy of *k*-mer Jaccard index

Here compute quantities required in task 4.

## Task 5: Jaccard index on real genomes

Here compute quantities required in task 5.

## Task 6: Accuracy of minimizers and minhash

Here compute quantities required in task 5. You can combine tasks 6 and 7 here to run long computations only once.

## Task 7: Running time and memory